In [2]:
!pip install keras==2.3.1
!pip install tensorflow==2.2
import keras
import numpy as np
import pandas as pd
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot

from mpl_toolkits.mplot3d import Axes3D

# define the standalone discriminator model
def define_discriminator(n_inputs=3):#n_inputs is the number of features, 3 in this case.
    model = Sequential()
    model.add(Dense(25, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))    
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator(latent_dim, n_outputs=3):#n_outputs is the number of features, 3 in this case
    model = Sequential()
    model.add(Dense(15, activation='relu', kernel_initializer='he_uniform', input_dim=latent_dim))    
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(n_outputs, activation='linear'))
    return model

def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

# generate n real samples with class labels
def generate_real_samples(n):
    # generate inputs in [-0.5, 0.5]
    X1=(rand(n)-0.5)*6*np.pi
    X2=(rand(n)-0.5)*4*np.pi
    #X1 = rand(n) - 0.5
    # generate outputs X^2
    X3=np.sin(X1+X2)
    #X2 = X1 * X1
    # stack arrays
    X1 = X1.reshape(n, 1)
    X2 = X2.reshape(n, 1)
    X3 = X3.reshape(n,1)
    X = hstack((X1, X2, X3))
    # generate class labels
    y = ones((n, 1))
    return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n):
    # generate points in the latent space
    x_input = randn(latent_dim * n)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n, latent_dim)
    return x_input

def generate_fake_samples(generator, latent_dim, n):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n)
    # predict outputs
    X = generator.predict(x_input)
    # create class labels
    y = zeros((n, 1))
    return X, y

x_real, y_real=generate_real_samples(20)
df_real=pd.DataFrame({'x' : x_real[:, 0], 'y' : x_real[:, 1],'z' : x_real[:,2]})
df_real.to_csv('real_data.csv', index=False)
# evaluate the discriminator and plot real and fake points
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):# n is the number of generated data (fake)
    # prepare real samples
    #x_real, y_real = generate_real_samples(20) #20 is the number of real data
    # evaluate discriminator on real examples
    _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(generator, latent_dim, n)
    # evaluate discriminator on fake examples
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print(epoch, acc_real, acc_fake)
    fig = pyplot.figure()
    ax = Axes3D(fig)
    ax.scatter(x_real[:, 0], x_real[:, 1], x_real[:,2], c='red')
    # scatter plot real and fake data points
    ax.scatter(x_fake[:, 0], x_fake[:, 1], x_fake[:,2], c='blue')
    #pyplot.scatter(x_real[:, 0], x_real[:, 1], color='red')
    #pyplot.scatter(x_fake[:, 0], x_fake[:, 1], color='blue')
    # save plot to file
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()
    
    #Store data in a csv file
    df = pd.DataFrame({'x' : x_fake[:, 0], 'y' : x_fake[:, 1],'z' : x_fake[:,2]})
    for i in range(epoch):
        cas=[2000,4000,6000,8000,10000]
        if i in cas:
            file='gen_data_%03d.csv' %i
            df.to_csv(file, index=False)
    
# train the generator and discriminator
# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=10000, n_batch=128,
    n_eval=2000):
    # determine half the size of one batch, for updating the discriminator
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # prepare real samples
        x_real, y_real = generate_real_samples(half_batch)
        # prepare fake examples
        x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # update discriminator
        d_model.train_on_batch(x_real, y_real)
        d_model.train_on_batch(x_fake, y_fake)
        # prepare points in latent space as input for the generator
        x_gan = generate_latent_points(latent_dim, n_batch)
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))
        # update the generator via the discriminator✬s error
        gan_model.train_on_batch(x_gan, y_gan)
        # evaluate the model every n_eval epochs
        if (i+1) % n_eval == 0:
            summarize_performance(i, g_model, d_model, latent_dim)
        
# size of the latent space
latent_dim = 5
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)

     |████████████████████████████████| 377 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 516.2 MB 3.4 kB/s eta 0:00:015    |█████▉                          | 94.3 MB 48.9 MB/s eta 0:00:09     |████████████▏                   | 196.8 MB 3.3 MB/s eta 0:01:38     |████████████████████████▌       | 394.6 MB 2.4 MB/s eta 0:00:52��████████████████▊       | 398.4 MB 2.4 MB/s eta 0:00:50  | 426.0 MB 2.3 MB/s eta 0:00:40��████████▏    | 438.7 MB 2.3 MB/s eta 0:00:34��████████▊    | 446.8 MB 2.3 MB/s eta 0:00:31��███████████████████▎   | 455.8 MB 2.3 MB/s eta 0:00:27.7 MB 2.5 MB/s eta 0:00:23    |█████████████████████████████▊  | 478.9 MB 2.5 MB/s eta 0:00:15B/s eta 0:00:10�█████████████████████████▎| 504.2 MB 2.7 MB/s eta 0:00:05
     |████████████████████████████████| 454 kB 65.6 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 6.4 MB/s  eta 0:00:01
     |████████████████████████████████| 2.9 MB 50.5 MB/s eta 0:00:01
     |████████████████████████████████| 26.1

Using TensorFlow backend.
/srv/conda/envs/notebook/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1999 0.44999998807907104 0.7699999809265137
3999 0.75 0.6700000166893005
5999 0.550000011920929 0.6499999761581421
7999 0.5 0.5
9999 0.550000011920929 0.6399999856948853
